In [1]:
import random
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import pandas as pd
#import imageio
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

import matplotlib.pyplot as plt
%matplotlib inline
import math

import torchvision
import torchvision.transforms as transforms
import optuna  #载入optuna优化包

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True,
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False,
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100,
                                          shuffle=False)


##optuna######
###耗费时间的过程。。。。。。。。。。。。。。。
# Define a basic convolutional layer
# 自定义的可学习ReLU激活函数：max(input, sigma*input)
class CustomReLU(nn.Module):
    def __init__(self, seqFlag, trial):
        super(CustomReLU, self).__init__()
        self.sigma = nn.Parameter(torch.tensor(trial.suggest_float(f'sigma_{seqFlag}', 0.1, 5.0)))

    def forward(self, x):
        return torch.max(x, self.sigma * x)



def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)


# 定义ResNet模型
class ResNet(nn.Module):
    def __init__(self, trial, num_classes=10):
        super(ResNet, self).__init__()
        self.conv1 = conv3x3(3, 16)
        self.bn1 = nn.BatchNorm2d(16)
        self.custom1 = CustomReLU(1, trial)  # 使用 CustomReLU

        # Layer 1
        self.layer1_conv1 = conv3x3(16, 16)
        self.layer1_bn1 = nn.BatchNorm2d(16)
        self.layer1_conv2 = conv3x3(16, 16)
        self.layer1_bn2 = nn.BatchNorm2d(16)
        self.layer1_extra_conv1 = conv3x3(16, 16)
        self.layer1_extra_bn1 = nn.BatchNorm2d(16)
        self.layer1_extra_conv2 = conv3x3(16, 16)
        self.layer1_extra_bn2 = nn.BatchNorm2d(16)
        self.gelub11 = CustomReLU(2, trial)
        self.gelub12 = CustomReLU(3, trial)
        self.gelub13 = CustomReLU(4, trial)
        self.gelub14 = CustomReLU(5, trial)

        # Layer 2
        self.layer2_conv1 = conv3x3(16, 32, stride=2)
        self.layer2_bn1 = nn.BatchNorm2d(32)
        self.layer2_conv2 = conv3x3(32, 32)
        self.layer2_bn2 = nn.BatchNorm2d(32)
        self.layer2_extra_conv1 = conv3x3(16, 32, stride=2)  # 调整residual的通道数
        self.layer2_extra_bn1 = nn.BatchNorm2d(32)
        self.layer2_extra_conv2 = conv3x3(32, 32)
        self.layer2_extra_bn2 = nn.BatchNorm2d(32)
        self.gelub21 = CustomReLU(6, trial)
        self.gelub22 = CustomReLU(7, trial)
        self.gelub23 = CustomReLU(8, trial)
        self.gelub24 = CustomReLU(9, trial)
        self.layer2_downsample = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=1, stride=2, bias=False),
            nn.BatchNorm2d(32)
        )

        # Layer 3
        self.layer3_conv1 = conv3x3(32, 64, stride=2)
        self.layer3_bn1 = nn.BatchNorm2d(64)
        self.layer3_conv2 = conv3x3(64, 64)
        self.layer3_bn2 = nn.BatchNorm2d(64)
        self.layer3_extra_conv1 = conv3x3(32, 64, stride=2)  # 调整residual的通道数
        self.layer3_extra_bn1 = nn.BatchNorm2d(64)
        self.layer3_extra_conv2 = conv3x3(64, 64)
        self.layer3_extra_bn2 = nn.BatchNorm2d(64)
        self.gelub31 = CustomReLU(10, trial)
        self.gelub32 = CustomReLU(11, trial)
        self.gelub33 = CustomReLU(12, trial)
        self.gelub34 = CustomReLU(13, trial)

        self.layer3_downsample = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=1, stride=2, bias=False),
            nn.BatchNorm2d(64)
        )

        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        # Layer 0
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.custom1(out)

        # Layer 1
        residual = out
        out = self.layer1_conv1(out)
        out = self.layer1_bn1(out)
        out = self.gelub11(out)
        out = self.layer1_conv2(out)
        out = self.layer1_bn2(out)
        out += residual
        out = self.gelub12(out)

        residual = out
        out = self.layer1_extra_conv1(out)
        out = self.layer1_extra_bn1(out)
        out = self.gelub13(out)
        out = self.layer1_extra_conv2(out)
        out = self.layer1_extra_bn2(out)
        out += residual
        out = self.gelub14(out)

        # Layer 2
        residual = out
        out = self.layer2_conv1(out)
        out = self.layer2_bn1(out)
        out = self.gelub21(out)
        out = self.layer2_conv2(out)
        out = self.layer2_bn2(out)
        out = self.gelub22(out)
        residual = self.layer2_downsample(residual)
        out += residual
        out = self.gelub22(out)

        residual = out
        out = self.layer2_extra_conv2(out)
        out = self.layer2_extra_bn2(out)
        out = self.gelub23(out)
        out = self.layer2_extra_conv2(out)
        out = self.layer2_extra_bn2(out)
        out += residual
        out = self.gelub24(out)

        # Layer 3
        residual = out
        out = self.layer3_conv1(out)
        out = self.layer3_bn1(out)
        out = self.gelub31(out)
        out = self.layer3_conv2(out)
        out = self.layer3_bn2(out)
        residual = self.layer3_downsample(residual)
        out += residual
        out = self.gelub32(out)

        residual = out
        out = self.layer3_extra_conv2(out)
        out = self.layer3_extra_bn2(out)
        out = self.gelub33(out)
        out = self.layer3_extra_conv2(out)
        out = self.layer3_extra_bn2(out)
        out += residual
        out = self.gelub34(out)

        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

    def forward(self, x):
        # Layer 0
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.custom1(out)

        # Layer 1
        residual = out
        out = self.layer1_conv1(out)
        out = self.layer1_bn1(out)
        out = self.gelub11(out)
        out = self.layer1_conv2(out)
        out = self.layer1_bn2(out)
        out += residual
        out = self.gelub12(out)

        residual = out
        out = self.layer1_extra_conv1(out)
        out = self.layer1_extra_bn1(out)
        out = self.gelub13(out)
        out = self.layer1_extra_conv2(out)
        out = self.layer1_extra_bn2(out)
        out += residual
        out = self.gelub14(out)

        # Layer 2
        residual = out
        out = self.layer2_conv1(out)
        out = self.layer2_bn1(out)
        out = self.gelub21(out)
        out = self.layer2_conv2(out)
        out = self.layer2_bn2(out)
        out = self.gelub22(out)
        residual = self.layer2_downsample(residual)
        out += residual
        out = self.gelub22(out)

        residual = out
        out = self.layer2_extra_conv2(out)
        out = self.layer2_extra_bn2(out)
        out = self.gelub23(out)
        out = self.layer2_extra_conv2(out)
        out = self.layer2_extra_bn2(out)
        out += residual
        out = self.gelub24(out)

        # Layer 3
        residual = out
        out = self.layer3_conv1(out)
        out = self.layer3_bn1(out)
        out = self.gelub31(out)
        out = self.layer3_conv2(out)
        out = self.layer3_bn2(out)
        residual = self.layer3_downsample(residual)
        out += residual
        out = self.gelub32(out)

        residual = out
        out = self.layer3_extra_conv2(out)
        out = self.layer3_extra_bn2(out)
        out = self.gelub33(out)
        out = self.layer3_extra_conv2(out)
        out = self.layer3_extra_bn2(out)
        out += residual
        out = self.gelub34(out)

        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


learning_rate = 0.001
num_epochs = 100


# 定义目标函数
def objective(trial):
    # 模型实例化
    model = ResNet(trial)
    model.to(device)
    curr_lr = learning_rate
    # 损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=curr_lr)  # 保持 Adam 优化器

    #history = {'train_loss': [], 'valid_loss': [], 'train_acc': [], 'valid_acc': []}

    for epoch in range(num_epochs):
        optimizer = torch.optim.Adam(model.parameters(), lr=curr_lr)  # 保持 Adam 优化器
        train_loss, train_correct = 0.0, 0
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
            scores, predictions = torch.max(outputs.data, 1)
            train_correct += (predictions == labels).sum().item()

        # Decay learning rate
        if (epoch + 1) % 20 == 0:
            curr_lr /= 3
            update_lr(optimizer, curr_lr)

        valid_loss, valid_correct = 0.0, 0
        model.eval()
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            valid_loss += loss.item() * images.size(0)
            scores, predictions = torch.max(outputs.data, 1)
            valid_correct += (predictions == labels).sum().item()
        ##不好的trial剪枝    
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        train_loss = train_loss / len(train_loader.sampler)
        train_acc = train_correct / len(train_loader.sampler) * 100
        valid_loss = valid_loss / len(test_loader.sampler)
        valid_acc = valid_correct / len(test_loader.sampler) * 100

        print("Epoch:{}/{} \t Train Loss:{:.4f} Valid Loss:{:.4f} \t Train Acc:{:.2f} %  Valid Acc:{:.2f} %".format(
            epoch + 1, num_epochs,
            train_loss,
            valid_loss,
            train_acc,
            valid_acc))
        # history['train_loss'].append(train_loss)
        # history['valid_loss'].append(valid_loss)
        # history['train_acc'].append(train_acc)
        # history['valid_acc'].append(valid_acc)

    return valid_acc


study = optuna.create_study(direction='maximize')

# 运行优化过程
study.optimize(objective, n_trials=20)

# 打印最佳参数和目标值
print('Best trial:')
print('  Value: ', study.best_trial.value)
print('  Params: ')
for key, value in study.best_trial.params.items():
    print('    {}: {}'.format(key, value))

Files already downloaded and verified


[I 2024-10-10 18:50:34,956] A new study created in memory with name: no-name-841e8774-1636-4949-833e-34fbcaf6f9bf


Epoch:1/100 	 Train Loss:1.7033 Valid Loss:1.9421 	 Train Acc:40.60 %  Valid Acc:39.38 %
Epoch:2/100 	 Train Loss:1.3813 Valid Loss:1.1364 	 Train Acc:50.66 %  Valid Acc:59.64 %
Epoch:3/100 	 Train Loss:1.1436 Valid Loss:1.0809 	 Train Acc:59.19 %  Valid Acc:62.98 %
Epoch:4/100 	 Train Loss:0.9934 Valid Loss:0.9518 	 Train Acc:64.92 %  Valid Acc:66.90 %
Epoch:5/100 	 Train Loss:0.8776 Valid Loss:0.7987 	 Train Acc:69.30 %  Valid Acc:72.40 %
Epoch:6/100 	 Train Loss:0.8070 Valid Loss:0.7858 	 Train Acc:71.87 %  Valid Acc:73.24 %
Epoch:7/100 	 Train Loss:0.7535 Valid Loss:0.7548 	 Train Acc:73.80 %  Valid Acc:75.30 %
Epoch:8/100 	 Train Loss:0.7046 Valid Loss:0.7146 	 Train Acc:75.49 %  Valid Acc:75.75 %
Epoch:9/100 	 Train Loss:0.6769 Valid Loss:0.6823 	 Train Acc:76.52 %  Valid Acc:77.73 %
Epoch:10/100 	 Train Loss:0.6430 Valid Loss:0.7296 	 Train Acc:77.85 %  Valid Acc:75.92 %
Epoch:11/100 	 Train Loss:0.6191 Valid Loss:0.6250 	 Train Acc:78.62 %  Valid Acc:79.12 %
Epoch:12/100 	 Trai

[I 2024-10-10 19:41:53,288] Trial 0 finished with value: 87.46000000000001 and parameters: {'sigma_1': 0.26742174393130047, 'sigma_2': 1.9779865491237294, 'sigma_3': 4.401853457300521, 'sigma_4': 1.3385868026779446, 'sigma_5': 2.527534694281982, 'sigma_6': 2.8527728581992395, 'sigma_7': 0.6888467192710976, 'sigma_8': 4.27951046995741, 'sigma_9': 4.878889308741704, 'sigma_10': 2.36192055963625, 'sigma_11': 3.571638090589973, 'sigma_12': 4.5912379715253415, 'sigma_13': 4.69426849157995}. Best is trial 0 with value: 87.46000000000001.


Epoch:100/100 	 Train Loss:0.1692 Valid Loss:0.4775 	 Train Acc:93.97 %  Valid Acc:87.46 %
Epoch:1/100 	 Train Loss:1.6692 Valid Loss:1.4244 	 Train Acc:40.92 %  Valid Acc:48.40 %
Epoch:2/100 	 Train Loss:1.4581 Valid Loss:1.2388 	 Train Acc:48.10 %  Valid Acc:55.38 %
Epoch:3/100 	 Train Loss:1.2436 Valid Loss:1.0686 	 Train Acc:55.29 %  Valid Acc:61.57 %
Epoch:4/100 	 Train Loss:1.1013 Valid Loss:0.9909 	 Train Acc:60.99 %  Valid Acc:64.71 %
Epoch:5/100 	 Train Loss:0.9808 Valid Loss:0.9032 	 Train Acc:65.38 %  Valid Acc:68.97 %
Epoch:6/100 	 Train Loss:0.8967 Valid Loss:0.8603 	 Train Acc:68.52 %  Valid Acc:70.47 %
Epoch:7/100 	 Train Loss:0.8315 Valid Loss:0.8150 	 Train Acc:70.97 %  Valid Acc:72.54 %
Epoch:8/100 	 Train Loss:0.7820 Valid Loss:0.7767 	 Train Acc:72.61 %  Valid Acc:73.04 %
Epoch:9/100 	 Train Loss:0.7404 Valid Loss:0.7898 	 Train Acc:74.07 %  Valid Acc:73.83 %
Epoch:10/100 	 Train Loss:0.6991 Valid Loss:0.7679 	 Train Acc:75.73 %  Valid Acc:73.56 %
Epoch:11/100 	 Tra

...

Epoch:100/100 	 Train Loss:0.1936 Valid Loss:0.4468 	 Train Acc:93.18 %  Valid Acc:87.57 %
Epoch:1/100 	 Train Loss:1.4829 Valid Loss:1.5747 	 Train Acc:45.74 %  Valid Acc:41.47 %
Epoch:2/100 	 Train Loss:1.2086 Valid Loss:1.0243 	 Train Acc:56.47 %  Valid Acc:62.88 %
Epoch:3/100 	 Train Loss:1.0842 Valid Loss:1.0044 	 Train Acc:61.26 %  Valid Acc:64.35 %
Epoch:4/100 	 Train Loss:0.9944 Valid Loss:0.9391 	 Train Acc:64.84 %  Valid Acc:66.53 %
Epoch:5/100 	 Train Loss:0.9172 Valid Loss:0.8736 	 Train Acc:67.58 %  Valid Acc:68.96 %
Epoch:6/100 	 Train Loss:0.8544 Valid Loss:0.7897 	 Train Acc:69.78 %  Valid Acc:72.13 %
Epoch:7/100 	 Train Loss:0.8130 Valid Loss:0.7860 	 Train Acc:71.34 %  Valid Acc:72.36 %
Epoch:8/100 	 Train Loss:0.7669 Valid Loss:0.7730 	 Train Acc:73.17 %  Valid Acc:73.52 %
Epoch:9/100 	 Train Loss:0.7208 Valid Loss:0.7063 	 Train Acc:74.67 %  Valid Acc:75.33 %
Epoch:10/100 	 Train Loss:0.6906 Valid Loss:0.7575 	 Train Acc:76.03 %  Valid Acc:74.31 %
Epoch:11/100 	 Tra

[I 2024-10-11 10:12:55,647] Trial 18 finished with value: 86.59 and parameters: {'sigma_1': 3.846388284561221, 'sigma_2': 0.2244692508604328, 'sigma_3': 1.7916029564669773, 'sigma_4': 3.788433303123839, 'sigma_5': 0.6941687585234804, 'sigma_6': 4.73891779224925, 'sigma_7': 3.575094652072449, 'sigma_8': 2.9043671189295686, 'sigma_9': 0.46813959014299344, 'sigma_10': 3.2386122678692795, 'sigma_11': 0.12125480215144287, 'sigma_12': 3.110229359182619, 'sigma_13': 1.936660242468178}. Best is trial 14 with value: 87.62.


Epoch:100/100 	 Train Loss:0.2217 Valid Loss:0.4559 	 Train Acc:92.12 %  Valid Acc:86.59 %
Epoch:1/100 	 Train Loss:1.5206 Valid Loss:1.4180 	 Train Acc:44.32 %  Valid Acc:50.27 %
Epoch:2/100 	 Train Loss:1.2767 Valid Loss:1.1112 	 Train Acc:54.23 %  Valid Acc:59.13 %
Epoch:3/100 	 Train Loss:1.1086 Valid Loss:1.0483 	 Train Acc:60.31 %  Valid Acc:63.11 %
Epoch:4/100 	 Train Loss:1.0115 Valid Loss:0.9170 	 Train Acc:64.17 %  Valid Acc:66.86 %
Epoch:5/100 	 Train Loss:0.9137 Valid Loss:0.9337 	 Train Acc:67.71 %  Valid Acc:67.52 %
Epoch:6/100 	 Train Loss:0.8280 Valid Loss:0.7960 	 Train Acc:71.00 %  Valid Acc:72.48 %
Epoch:7/100 	 Train Loss:0.7668 Valid Loss:0.7519 	 Train Acc:73.25 %  Valid Acc:74.53 %
Epoch:8/100 	 Train Loss:0.7254 Valid Loss:0.7442 	 Train Acc:74.80 %  Valid Acc:74.81 %
Epoch:9/100 	 Train Loss:0.6885 Valid Loss:0.6577 	 Train Acc:76.33 %  Valid Acc:77.02 %
Epoch:10/100 	 Train Loss:0.6579 Valid Loss:0.6937 	 Train Acc:77.26 %  Valid Acc:76.50 %
Epoch:11/100 	 Tra

[I 2024-10-11 11:01:42,578] Trial 19 finished with value: 85.98 and parameters: {'sigma_1': 2.381431423108764, 'sigma_2': 0.8504313175559848, 'sigma_3': 1.6278567410553464, 'sigma_4': 0.9521696120376688, 'sigma_5': 2.1942804886419536, 'sigma_6': 3.8088310388317494, 'sigma_7': 3.478954707992676, 'sigma_8': 2.088646083544198, 'sigma_9': 2.2508835739994337, 'sigma_10': 4.457826925471275, 'sigma_11': 0.30514728424852056, 'sigma_12': 2.9274708768758857, 'sigma_13': 3.0602939868679737}. Best is trial 14 with value: 87.62.


Epoch:100/100 	 Train Loss:0.2103 Valid Loss:0.4867 	 Train Acc:92.55 %  Valid Acc:85.98 %
Best trial:
  Value:  87.62
  Params: 
    sigma_1: 2.015878437710638
    sigma_2: 1.3699524863056682
    sigma_3: 1.0552197677721469
    sigma_4: 0.10853627608883176
    sigma_5: 0.17723314275762636
    sigma_6: 4.135343301696687
    sigma_7: 2.158802562492464
    sigma_8: 3.085411065052954
    sigma_9: 1.8212684771670142
    sigma_10: 3.543530557481457
    sigma_11: 3.3671782854477574
    sigma_12: 4.011143417997994
    sigma_13: 2.3495644512839267
